In [1]:
from newsapi import NewsApiClient

In [2]:
import requests
import json

In [3]:
from string import Template

In [4]:
from bs4 import *

In [5]:
from selenium import webdriver

In [6]:
import time

In [ ]:
url = ("""https://newsapi.org/v2/everything?
language=en
&from=2019-10-01
&sources=cnbc
&apiKey=749e4a9b92e046a386a4420df0de07ee""")

requests.get(url)


In [ ]:
url_template = Template("""
https://newsapi.org/v2/everything?\
language=$language\
&from=$from\
&sources=$sources\
&apiKey=$apiKey\
&pageSize=$pageSize\
&page=$page\
""")

In [ ]:
url = url_template.substitute({"language": "en", "from": "2019-10-01", "sources": "cnbc", 
                               "apiKey": "749e4a9b92e046a386a4420df0de07ee", "pageSize": 30, "page": 2})

In [ ]:
url

In [ ]:
response = requests.get(url)

In [ ]:
json_response=response.json()

In [ ]:
len(json_response["articles"])

In [ ]:
for entity in json_response:
    print(entity)

In [ ]:
news_articles=[]
for entity in json_response:
    if(entity=='articles'):
        for i in range(0,len(json_response[entity])):
            news_articles.append(json_response['articles'][i]['content'])     

In [ ]:
news_articles[0]

In [ ]:
len(news_articles)

In [ ]:
# Selenium web driver.
driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver")

In [11]:
# Function to get and store CNBC news articles (text and JSON) in respective directories.

def get_news_articles_cnbc (from_date="2019-10-01", api_key="749e4a9b92e046a386a4420df0de07ee", page_size=100, pages=3,
                            start_page=1, json_data_dir="C:/", text_data_dir="C:/",
                            chrome_driver="C:/Users/Administrator/chromedriver_win32/chromedriver"):
    # Create the URL template to query News API.
    url_template = Template("""
    https://newsapi.org/v2/everything?\
    language=$language\
    &from=$from\
    &sources=$sources\
    &apiKey=$apiKey\
    &pageSize=$pageSize\
    &page=$page\
    """)
    # Get the driver object.
    driver = webdriver.Chrome(chrome_driver)
    
    # Iterate over a number of pages.
    for page in range(start_page, pages):
        url = url_template.substitute({"language": "en", "from": from_date, "sources": "cnbc", 
                                       "apiKey": api_key, "pageSize": page_size, "page": page})
        response = requests.get(url)
        print(response.json())
        # Iterate over page size, as the number of articles retrieved is = page size.
        for article in range(1, page_size):
            # File name - Article + underscore + time.
            news_article_fname = "article_" + time.strftime("%Y%m%d_%H%M%S")
            # Article - In text and JSON formats.
            news_article_text = ""
            news_article_json = ""
            print(response.json().keys())
            # Check if articles are fetched, if not - continue.
            if ("articles" in response.json().keys()):
                # Get article URL. We receive this by parsing the News API response.
                news_url = response.json()["articles"][article]["url"]
                # Print the current status.
                print("INFO: Fetching article - " + str(article) + " of page - " + str(page) + ". URL - " + news_url)
                # Get page using the Chromium driver.
                news_article = driver.get(news_url)
                # From this page, get the inner HTML. This gets the entire page as one HTML Document.
                news_article_html = driver.find_element_by_xpath("//*").get_attribute("innerHTML")
                # Beautiful soup parsed.
                news_article_bs = BeautifulSoup(news_article_html, "lxml")
                # For CNBC, the structure is - one div with id = MainContent, and that consists of divs in class - group.
                news_article_divs = news_article_bs.find(id="MainContent").find_all("div", class_="group")
                # Iterate over the divs to fetch and concatenate paragraphs.
                for div in news_article_divs:
                    # Get all paragraphs in that div.
                    news_article_p = div.find_all("p")
                    for p in news_article_p:
                        # Concatenate the text string with the new data from the paragraph tag of the news article.
                        news_article_text = news_article_text + "\n" + p.get_text()
                # If the text is empty, don't dump to file.
                if not (news_article_text.strip()==""):
                    # Write JSON, using JSON.DUMPS function.
                    news_article_jfile = open(json_data_dir + "/" + news_article_fname + ".json", "w+", encoding="utf-8")
                    news_article_jfile.write(json.dumps(response.json()["articles"][article]))
                    news_article_jfile.close()
                    # Write Text.
                    news_article_tfile = open(text_data_dir + "/" + news_article_fname + ".txt", "w+", encoding="utf-8")
                    news_article_tfile.write(news_article_text.strip())
                    news_article_tfile.close()
    # Close Selenium Chrome driver.
    driver.close()

In [38]:
# Function to get and store CNBC news articles (text and JSON) in respective directories.

def get_news_articles_bi (from_date="2019-10-01", api_key="749e4a9b92e046a386a4420df0de07ee", page_size=20, pages=10,
                            start_page=1, json_data_dir="C:/", text_data_dir="C:/",
                            chrome_driver="C:/Users/Administrator/chromedriver_win32/chromedriver"):
    # Create the URL template to query News API.
    url_template = Template("""
    https://newsapi.org/v2/everything?\
    language=$language\
    &from=$from\
    &sources=$sources\
    &apiKey=$apiKey\
    &pageSize=$pageSize\
    &page=$page\
    """)
    # Get the driver object.
    driver = webdriver.Chrome(chrome_driver)
    
    # Iterate over a number of pages.
    for page in range(start_page, pages):
        url = url_template.substitute({"language": "en", "from": from_date, "sources": "business-insider", 
                                       "apiKey": api_key, "pageSize": page_size, "page": page})
        response = requests.get(url)
        # Iterate over page size, as the number of articles retrieved is = page size.
        for article in range(1, page_size):
            # File name - Article + underscore + time.
            news_article_fname = "article_" + time.strftime("%Y%m%d_%H%M%S")
            # Article - In text and JSON formats.
            news_article_text = ""
            news_article_json = ""
            # Check if articles are fetched, if not - continue.
            if ("articles" in response.json().keys()):
                # Get article URL. We receive this by parsing the News API response.
                news_url = response.json()["articles"][article]["url"]
                # Print the current status.
                print("INFO: Fetching article - " + str(article) + " of page - " + str(page) + ". URL - " + news_url)
                # Get page using the Chromium driver.
                news_article = driver.get(news_url)
                # From this page, get the inner HTML. This gets the entire page as one HTML Document.
                news_article_html = driver.find_element_by_xpath("//*").get_attribute("innerHTML")
                # Beautiful soup parsed.
                news_article_bs = BeautifulSoup(news_article_html, "lxml")
                # For CNBC, the structure is - one div with id = MainContent, and that consists of divs in class - group.
                news_article_divs = news_article_bs.find(id="the_bi_content")
                # If the divs is None, then continue with the loop without parsing.
                if (news_article_divs is None):
                    continue
                news_article_divs = news_article_divs.find_all("div")
                # Iterate over the divs to fetch and concatenate paragraphs.
                for div in news_article_divs:
                    # Get all paragraphs in that div.
                    news_article_p = div.find_all("p")
                    for p in news_article_p:
                        # Concatenate the text string with the new data from the paragraph tag of the news article.
                        news_article_text = news_article_text + "\n" + p.get_text()
                # If the text is empty, don't dump to file.
                if not (news_article_text.strip()==""):
                    # Write JSON, using JSON.DUMPS function.
                    news_article_jfile = open(json_data_dir + "/" + news_article_fname + ".json", "w+", encoding="utf-8")
                    news_article_jfile.write(json.dumps(response.json()["articles"][article]))
                    news_article_jfile.close()
                    # Write Text.
                    news_article_tfile = open(text_data_dir + "/" + news_article_fname + ".txt", "w+", encoding="utf-8")
                    news_article_tfile.write(news_article_text.strip())
                    news_article_tfile.close()
    # Close Selenium Chrome driver.
    driver.close()

In [ ]:
get_news_articles_cnbc(json_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/JSON",
                       text_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/Text", start_page=2)

In [39]:
get_news_articles_bi(json_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/JSON",
                       text_data_dir="C:/Users/Administrator/Documents/GitHub/newsreels/data/News/Text", start_page=2)

INFO: Fetching article - 1 of page - 2. URL - https://www.businessinsider.com/house-democrats-weigh-masking-whistleblower-identity-trump-allies-2019-10
INFO: Fetching article - 2 of page - 2. URL - https://www.businessinsider.com/trump-trade-japan-and-us-sign-limited-trade-agreement-2019-10
INFO: Fetching article - 3 of page - 2. URL - https://www.businessinsider.com/story-of-navy-donald-cook-unsafe-incident-with-russian-fighters-2019-10
INFO: Fetching article - 4 of page - 2. URL - https://www.businessinsider.com/disney-plus-price
INFO: Fetching article - 5 of page - 2. URL - https://www.businessinsider.com/orange-county-purest-drinking-water-filtered-sewage-2019-10
INFO: Fetching article - 6 of page - 2. URL - https://www.businessinsider.com/jacoby-brissett-run-it-down-their-throats-colts-upset-chiefs-2019-10
INFO: Fetching article - 7 of page - 2. URL - https://www.businessinsider.com/can-you-watch-videos-on-apple-watch
INFO: Fetching article - 8 of page - 2. URL - https://www.busin

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [ ]:
#Text Format
TRITTextResponse = TRITResponse.text
#print(TRITTextResponse)

#Json Format
TRITJsonResponse = json.loads(TRITTextResponse)
print(TRITJsonResponse)

for entity in TRITJsonResponse:
    for info in TRITJsonResponse[entity]:
        topics={}
        geography={}
        organizations={}
        persons={}
        industries={}
        currencies={}
        market_index={}
        
        #Topics
        if(info=='_typeGroup' and str(TRITJsonResponse[entity][info])=='topics'):
            for companyinfo in (TRITJsonResponse[entity]):
                if(companyinfo in ['_type','name','confidencelevel','score','relevance','permid']):
                    topics[companyinfo]=str(TRITJsonResponse[entity][companyinfo])
        if topics:
            topics_list.append(topics)

        
        #Geographical Locations
        if((info=='_type' and str(TRITJsonResponse[entity][info]) in ['Country','City','Continent','Region','ProvinceOrState'])):
            for companyinfo in (TRITJsonResponse[entity]):
                if(companyinfo in ['_type','name','confidencelevel','score','relevance','permid']):
                    geography[companyinfo]=str(TRITJsonResponse[entity][companyinfo])
                if(companyinfo=='resolutions'):
                    if('permid' in TRITJsonResponse[entity][companyinfo][0]):  
                        geography['permid']=str(TRITJsonResponse[entity][companyinfo][0]['permid'])
                    if('id' in TRITJsonResponse[entity][companyinfo][0]):  
                        geography['id']=str(TRITJsonResponse[entity][companyinfo][0]['id'])
                    if('score' in TRITJsonResponse[entity][companyinfo][0]):  
                        geography['score']=str(TRITJsonResponse[entity][companyinfo][0]['score'])
                    if('ticker' in TRITJsonResponse[entity][companyinfo][0]):
                        geography['ticker']=str(TRITJsonResponse[entity][companyinfo][0]['ticker'])
        if geography:
            geographical_list.append(geography)
